# Transfer Learning

In [ ]:
%pylab inline

In [1]:
import re

from bigdl.nn.criterion import CrossEntropyCriterion
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType

from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Dense, Input, Flatten
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
from glob import glob



Adding /usr/lib64/python2.7/site-packages/bigdl/share/lib/bigdl-0.6.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /usr/lib64/python2.7/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
Adding /usr/lib/python2.7/site-packages/zoo/share/lib/analytics-zoo-bigdl_0.6.0-spark_2.1.0-0.2.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /usr/lib/python2.7/site-packages/zoo/share/conf/spark-analytics-zoo.conf to sys.path


/usr/lib64/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/lib64/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:169: RuntimeWarning: numpy

creating: createDefault
creating: createSGD
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor
creating: createSeqToTensor


In [234]:
from bigdl.nn.criterion import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType, ArrayType
from zoo.common.nncontext import *
from zoo.feature.image import *
from zoo.pipeline.api.keras.layers import Input, Flatten, Dense
from zoo.pipeline.api.keras.models import *
from zoo.pipeline.api.net import *
from zoo.pipeline.nnframes import *
from zoo.feature.image.imagePreprocessing import *

import random
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from zoo.pipeline.api.keras.metrics import AUC
from pyspark.sql.types import *

import numpy as np
from sklearn.metrics import roc_auc_score
from pyspark.ml import Pipeline
from bigdl.optim.optimizer import * 

/usr/lib64/python2.7/site-packages/sklearn/utils/__init__.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/usr/lib64/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/usr/lib64/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/usr/lib64/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/usr/lib64/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _el

In [148]:
sparkConf = SparkConf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)

In [235]:
sparkConf = create_spark_conf().setAppName("testMutipleLabels")
sc = init_nncontext(sparkConf)
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [149]:
path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/red_shirt/"
print path
new_path="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/"
print new_path

/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/red_shirt/
/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/


In [144]:
type(sc)

pyspark.context.SparkContext

In [145]:
from bigdl.util.common import *
from bigdl.transform.vision.image import *
import matplotlib.pyplot as plt

manually set model_path and image_path for training

1. model_path = path to the pre-trained models. (E.g. path/to/model/bigdl_inception-v1_imagenet_0.4.0.model)

2. image_path = path to the folder of the training images. (E.g. path/to/data/dogs-vs-cats/demo/\*/\*)

In [52]:
model_path = "/home/mahmood/analytics-zoo/demo/bigdl_inception-v1_imagenet_0.4.0.model"

#image_path = "file:///home/mahmood/analytics-zoo/demo/*/*"
#image_path=ImageFrame.read("hdfs:///datasets/xray/train/*", sc, 2) this does not work 
#image_path=ImageFrame.read("hdfs://hinfra/datasets/xray/train/*",sc,2)
#image_path = "file:///home/mahmood/Downloads/*/*"
#image_path = "file:///home/mahmood/analytics-zoo/demo/*/*"
image_path = "file:///home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/"
imageDF = NNImageReader.readImages(image_path, sc)

In [53]:
image_path

'file:///home/mahmood/analytics-zoo/apps/dogs-vs-cats/allJeansImage/*'

In [163]:
from glob import glob

txtfiles =[]
Image_index=[]
for file in glob("/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/*"):
    txtfiles.append(file.split('/'))
for i in range(len(txtfiles)):  
    Image_index.append(txtfiles[i][8:9])
    
print len(txtfiles)

1901


In [218]:
Labels1=[]
for j in range(len(Image_index)):
    s=re.sub('[^A-Za-z_.]', '', str(Image_index[j]))
    #print s
    s=s.strip('_.jpg')
    s=s.strip('_.jpeg')
    s=s.strip('_.JPG')
    s=s.strip('_.pn')
    Labels1.append(s)
Labels1[0:10]
    

['black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans',
 'black_jeans']

In [215]:
type(s)

str

In [137]:
label=Image_index

In [226]:
Img_indx=[]
for j in range(len(Image_index)):
    s=re.sub('[^A-Za-z0-9_.]', '', str(Image_index[j]))
    #print s
    
    Img_indx.append(s)
Img_indx[0:10]

['black_jeans_00000000.jpg',
 'black_jeans_00000001.jpeg',
 'black_jeans_00000002.jpeg',
 'black_jeans_00000003.jpg',
 'black_jeans_00000004.jpg',
 'black_jeans_00000005.jpeg',
 'black_jeans_00000006.jpg',
 'black_jeans_00000007.jpg',
 'black_jeans_00000008.jpeg',
 'black_jeans_00000009.jpg']

In [222]:
import csv
from itertools import izip

with open('AllLabels.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Img_indx,Labels1))

In [223]:

 
# open file
with open('AllLabels.csv', 'rb') as f:
    reader = csv.reader(f)
 
    # read file row by row
    for row in reader:
        print row

['black_jeans_00000000.jpg', 'black_jeans']
['black_jeans_00000001.jpeg', 'black_jeans']
['black_jeans_00000002.jpeg', 'black_jeans']
['black_jeans_00000003.jpg', 'black_jeans']
['black_jeans_00000004.jpg', 'black_jeans']
['black_jeans_00000005.jpeg', 'black_jeans']
['black_jeans_00000006.jpg', 'black_jeans']
['black_jeans_00000007.jpg', 'black_jeans']
['black_jeans_00000008.jpeg', 'black_jeans']
['black_jeans_00000009.jpg', 'black_jeans']
['black_jeans_00000010.jpg', 'black_jeans']
['black_jeans_00000011.jpg', 'black_jeans']
['black_jeans_00000013.jpg', 'black_jeans']
['black_jeans_00000014.jpg', 'black_jeans']
['black_jeans_00000015.jpg', 'black_jeans']
['black_jeans_00000016.jpg', 'black_jeans']
['black_jeans_00000017.jpg', 'black_jeans']
['black_jeans_00000018.jpg', 'black_jeans']
['black_jeans_00000019.jpg', 'black_jeans']
['black_jeans_00000020.jpg', 'black_jeans']
['black_jeans_00000021.jpg', 'black_jeans']
['black_jeans_00000022.jpg', 'black_jeans']
['black_jeans_00000023.jpg',

['blue_shirt_00000222.jpg', 'blue_shirt']
['blue_shirt_00000223.jpg', 'blue_shirt']
['blue_shirt_00000224.jpg', 'blue_shirt']
['blue_shirt_00000226.jpg', 'blue_shirt']
['blue_shirt_00000227.jpg', 'blue_shirt']
['blue_shirt_00000228.jpeg', 'blue_shirt']
['blue_shirt_00000229.jpg', 'blue_shirt']
['blue_shirt_00000230.jpg', 'blue_shirt']
['blue_shirt_00000231.jpg', 'blue_shirt']
['blue_shirt_00000232.jpg', 'blue_shirt']
['blue_shirt_00000233.jpg', 'blue_shirt']
['blue_shirt_00000234.jpg', 'blue_shirt']
['blue_shirt_00000235.jpg', 'blue_shirt']
['blue_shirt_00000236.jpg', 'blue_shirt']
['blue_shirt_00000237.jpg', 'blue_shirt']
['blue_shirt_00000238.jpg', 'blue_shirt']
['blue_shirt_00000240.png', 'blue_shirt']
['blue_shirt_00000241.jpg', 'blue_shirt']
['blue_shirt_00000242.jpg', 'blue_shirt']
['blue_shirt_00000243.jpg', 'blue_shirt']
['blue_shirt_00000244.jpg', 'blue_shirt']
['blue_shirt_00000245.jpg', 'blue_shirt']
['blue_shirt_00000246.png', 'blue_shirt']
['blue_shirt_00000247.jpeg', 'blu

In [237]:
image_path='/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/'
label_path = '/home/mahmood/analytics-zoo/apps/dogs-vs-cats/AllLabels.csv'

In [238]:
getLabel = udf(lambda x: text_to_label(x), DoubleType())

In [266]:
import pandas as pd
labelDF=pd.read_csv(label_path)

In [267]:
labelDF.head()

,Image_index,labels
0,black_jeans_00000000.jpg,black_jeans
1,black_jeans_00000001.jpeg,black_jeans
2,black_jeans_00000002.jpeg,black_jeans
3,black_jeans_00000003.jpg,black_jeans
4,black_jeans_00000004.jpg,black_jeans


In [271]:
sqlCtx = SQLContext(sc)
labelDFsp = sqlCtx.createDataFrame(labelDF)

In [292]:
label_texts = list("""black,jeans,blue,dress,shirt,red""".replace("\n", "").split(","))
print label_texts

label_map = {k: v for v, k in enumerate(label_texts)}
print label_map

['black', 'jeans', 'blue', 'dress', 'shirt', 'red']
{'blue': 2, 'shirt': 4, 'jeans': 1, 'black': 0, 'dress': 3, 'red': 5}


In [293]:
def text_to_label(text):
    arr = [0.0] * len(label_texts)
    for l in text.split("_"):
        arr[label_map[l]] = 1.0
     
    return arr
print text_to_label("black_jeans")   

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [300]:
getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))

In [301]:
labelDF11 = labelDFsp.select("Image_index", "labels") \
   .withColumn("label", getLabel(col('labels')))



In [302]:
labelDF11.select("Image_index","label").show()

+--------------------+--------------------+
|         Image_index|               label|
+--------------------+--------------------+
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0, 0.0, 0...|
|black_jeans_00000...|[1.0, 1.0,

In [306]:
#getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))
getName = udf(lambda row: os.path.basename(row[0]), StringType())
imageDF = NNImageReader.readImages(image_path, sc, resizeH=300, resizeW=300 ) \
    .withColumn("Image_index", getName(col('image')))


labelDF = spark.read.load(label_path, format="csv", sep=",", inferSchema="true", header="true") \
   .select("Image Index", "Finding Labels") \
   .withColumn("label", getLabel(col('Finding Labels')))

In [304]:
imageDF.head(1)

[Row(image=Row(origin=u'file:/home/mahmood/analytics-zoo/apps/dogs-vs-cats/JeansDatasets/allJeansImage/black_jeans_00000000.jpg', height=300, width=300, nChannels=3, mode=16, data=bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01,\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\

# MY code start from here

label_texts = list("""Atelectasis, Consolidation, Infiltration, Pneumothorax, Edema, Emphysema, Fibrosis, Effusion, Pneumonia, Pleural_Thickening, Cardiomegaly, Nodule, Mass, Hernia, No Finding""".replace("\n", "").split(", "))
label_map = {k: v for v, k in enumerate(label_texts)}

#-------------------------------------------
def text_to_label(text):
    arr = [0.0] * len(label_texts)
    
    for l in text.split(","):
        arr[label_map[l]] = 1.0
     
    return arr
#------------------------------------------
getLabel = udf(lambda x: text_to_label(x), ArrayType(DoubleType()))
getName = udf(lambda row: os.path.basename(row[0]), StringType())
imageDF = NNImageReader.readImages(image_path, sc, resizeH=256, resizeW=256) \
    .withColumn("Image Index", getName(col('image')))
labelDF = spark.read.load(label_path, format="csv", sep=",", inferSchema="true", header="true") \
   .select("Image Index", "Finding Labels") \
   .withColumn("label", getLabel(col('Finding Labels')))

In [307]:
train_df = imageDF.join(labelDF11, on="Image_index", how="inner")
(trainingDF, validationDF) = train_df.randomSplit([0.7, 0.3])

In [ ]:
(trainingDF, validationDF) = labelDF.randomSplit([0.9, 0.1])
labelDF.select("name","label").show(20)

In [308]:

type(trainingDF)


pyspark.sql.dataframe.DataFrame

## Fine-tune a pre-trained model

We fine-tune a pre-trained model by removing the last few layers, freezing the first few layers, and adding some new layers.

In [309]:
transformer = ChainedPreprocessing(
        [RowToImageFeature(), ImageResize(256, 256), ImageCenterCrop(224, 224),
         ImageChannelNormalize(123.0, 117.0, 104.0), ImageMatToTensor(), ImageFeatureToTensor()])

creating: createRowToImageFeature
creating: createImageResize
creating: createImageCenterCrop
creating: createImageChannelNormalize
creating: createImageMatToTensor
creating: createImageFeatureToTensor
creating: createChainedPreprocessing


### Load a pre-trained model

We use the Net API to load a pre-trained model, including models saved by Analytics Zoo, BigDL, Torch, Caffe and Tensorflow. Please refer to [Net API Guide](https://analytics-zoo.github.io/master/#APIGuide/PipelineAPI/net/).

In [310]:
full_model = Net.load_bigdl(model_path)

### Remove the last few layers

Here we print all the model layers and you can choose which layer(s) to remove.

When a model is loaded using Net, we can use the newGraph(output) api to define a Model with the output specified by the parameter. 

In [311]:
for layer in full_model.layers:
    print (layer.name())
model = full_model.new_graph(["pool5/drop_7x7_s1"])

data
conv1/7x7_s2
conv1/relu_7x7
pool1/3x3_s2
pool1/norm1
conv2/3x3_reduce
conv2/relu_3x3_reduce
conv2/3x3
conv2/relu_3x3
conv2/norm2
pool2/3x3_s2
inception_3a/3x3_reduce
inception_3a/5x5_reduce
inception_3a/relu_3x3_reduce
inception_3a/relu_5x5_reduce
inception_3a/pool
inception_3a/1x1
inception_3a/3x3
inception_3a/5x5
inception_3a/pool_proj
inception_3a/relu_pool_proj
inception_3a/relu_5x5
inception_3a/relu_3x3
inception_3a/relu_1x1
inception_3a/output
inception_3b/3x3_reduce
inception_3b/5x5_reduce
inception_3b/relu_3x3_reduce
inception_3b/relu_5x5_reduce
inception_3b/pool
inception_3b/1x1
inception_3b/3x3
inception_3b/5x5
inception_3b/pool_proj
inception_3b/relu_pool_proj
inception_3b/relu_5x5
inception_3b/relu_3x3
inception_3b/relu_1x1
inception_3b/output
pool3/3x3_s2
inception_4a/3x3_reduce
inception_4a/5x5_reduce
inception_4a/relu_3x3_reduce
inception_4a/relu_5x5_reduce
inception_4a/pool
inception_4a/1x1
inception_4a/3x3
inception_4a/5x5
inception_4a/pool_proj
inception_4a/relu_

The returning model's output layer is "pool5/drop_7x7_s1".

### Freeze some layers

We freeze layers from input to pool4/3x3_s2 inclusive.

model.freeze_up_to(["pool4/3x3_s2"])

In [324]:
inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(6, activation="sigmoid")(flatten)
lrModel = Model(inputNode, logits)

creating: createZooKerasInput
creating: createZooKerasFlatten
creating: createZooKerasDense
creating: createZooKerasModel


### Add a few new layers

In [313]:
val_summary = ValidationSummary(log_dir="/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/log", app_name="testMutipleLabels")

creating: createValidationSummary


inputNode = Input(name="input", shape=(3, 224, 224))
inception = model.to_keras()(inputNode)
flatten = Flatten()(inception)
logits = Dense(2)(flatten)
lrModel = Model(inputNode, logits)
classifier = NNClassifier(lrModel, CrossEntropyCriterion(), transformer) \
        .setLearningRate(0.003).setBatchSize(24).setMaxEpoch(1).setFeaturesCol("image") \
        .setCachingSample(False)
pipeline = Pipeline(stages=[classifier])


In [325]:
classifier = NNEstimator(lrModel, MultiLabelSoftMarginCriterion(), transformer, SeqToTensor([6])) \
     .setLearningRate(0.001).setBatchSize(24).setMaxEpoch(10).setFeaturesCol("image")\
     .setCachingSample(False)\
     .setValidation(EveryEpoch(), validationDF,[AUC()],  24)
#\
 #    .setValidationSummary(val_summary)
#\
    # .setCheckpoint("/home/mahmood/analytics-zoo/apps/dogs-vs-cats/logDirectroy/checkpoint", EveryEpoch(),False)

creating: createMultiLabelSoftMarginCriterion
creating: createSeqToTensor
creating: createFeatureLabelPreprocessing
creating: createNNEstimator
creating: createEveryEpoch
creating: createAUC


In [326]:
nnModel = classifier.fit(trainingDF)
print("Finished training")

creating: createToTuple
creating: createChainedPreprocessing
Finished training


In [327]:
nnModel.transform(trainingDF).show(5)

predictionDF = nnModel.transform(validationDF).cache()

predictionDF.select("Image_index","label","prediction").show(5)

+--------------------+--------------------+-----------+--------------------+--------------------+
|         Image_index|               image|     labels|               label|          prediction|
+--------------------+--------------------+-----------+--------------------+--------------------+
|black_jeans_00000...|[file:/home/mahmo...|black_jeans|[1.0, 1.0, 0.0, 0...|[0.61244214, 0.04...|
|black_jeans_00000...|[file:/home/mahmo...|black_jeans|[1.0, 1.0, 0.0, 0...|[0.8337579, 0.201...|
|blue_dress_000000...|[file:/home/mahmo...| blue_dress|[0.0, 0.0, 1.0, 1...|[0.6698415, 0.462...|
|blue_dress_000002...|[file:/home/mahmo...| blue_dress|[0.0, 0.0, 1.0, 1...|[0.7162987, 0.196...|
|blue_jeans_000001...|[file:/home/mahmo...| blue_jeans|[0.0, 1.0, 1.0, 0...|[0.69575536, 0.02...|
+--------------------+--------------------+-----------+--------------------+--------------------+
only showing top 5 rows

+--------------------+--------------------+--------------------+
|         Image_index|      

# Train the model

The transfer learning can finish in a few minutes. 

In [ ]:
predictionDF.select("name","label","prediction").sort("label", ascending=False).show(10)
predictionDF.select("name","label","prediction").show(10)
correct = predictionDF.filter("label=prediction").count()
overall = predictionDF.count()
accuracy = correct * 1.0 / overall
print("Test Error = %g " % (1.0 - accuracy))

As we can see, the model from transfer learning can achieve over 95% accuracy on the validation set.

## Visualize result

We randomly select some images to show, and print the prediction results here. 

cat: prediction = 1.0
dog: prediction = 2.0

In [ ]:
samplecat=predictionDF.filter(predictionDF.prediction==1.0).limit(3).collect()
sampledog=predictionDF.filter(predictionDF.prediction==2.0).sort("label", ascending=False).limit(3).collect()

In [ ]:
from IPython.display import Image, display
for cat in samplecat:
    print ("prediction:"), cat.prediction
    display(Image(cat.image.origin[5:]))

In [ ]:
for dog in sampledog:
    print ("prediction:"), dog.prediction
    display(Image(dog.image.origin[5:]))